# Notebook Imports

In [34]:
import os
import numpy as np
import tensorflow as tf

from time import strftime

# Constants

In [35]:
X_TRAIN_PATH = 'data/digit_xtrain.csv'
X_TEST_PATH = 'data/digit_xtest.csv'
Y_TRAIN_PATH = 'data/digit_ytrain.csv'
Y_TEST_PATH = 'data/digit_ytest.csv'

NR_CLASSES = 10
VALIDATION_SIZE = 10000
IMAGE_WIDTH = 28
IMAGE_HEIGHT = 28
CHANNELS = 1
TOTAL_INPUTS = IMAGE_HEIGHT * IMAGE_WIDTH * CHANNELS

LOGGING_PATH = 'tensorboard_mnist_digit_logs/'

In [36]:
x_train_all = np.loadtxt(X_TRAIN_PATH, delimiter=',', dtype=int)
y_train_all = np.loadtxt(Y_TRAIN_PATH, delimiter=',', dtype=int)
x_test = np.loadtxt(X_TEST_PATH, delimiter=',', dtype=int)
y_test = np.loadtxt(Y_TEST_PATH, delimiter=',', dtype=int)

# Preprocessing

In [37]:
# Re-scale
x_train_all, x_test = x_train_all / 255.0, x_test / 255.0

In [38]:
# Convert target values to one-hot encoding
y_train_all = np.eye(NR_CLASSES)[y_train_all]
y_test = np.eye(NR_CLASSES)[y_test]

In [39]:
# Create validation dataset from training data
x_val = x_train_all[:VALIDATION_SIZE]
y_val = y_train_all[:VALIDATION_SIZE]

x_train = x_train_all[VALIDATION_SIZE:]
y_train = y_train_all[VALIDATION_SIZE:]

# Setup Tensorflow Graph

In [40]:
x = tf.placeholder(tf.float32, shape=[None, TOTAL_INPUTS], name='x')
y = tf.placeholder(tf.float32, shape=[None, NR_CLASSES], name='labels')

AttributeError: module 'tensorflow' has no attribute 'placeholder'

### Neural Network Architecture

In [8]:
# Hyperparameters
nr_epochs = 50
learning_rate = 1e-4

n_hidden1 = 512
n_hidden2 = 64

In [9]:
def setup_layer(_input, weight, bias, name):
    with tf.name_scope(name):
        initial_w = tf.truncated_normal(shape=weight, stddev=0.1, seed=42)
        w = tf.Variable(initial_value=initial_w, name='W')

        initial_b = tf.constant(value=0.0, shape=bias)
        b = tf.Variable(initial_value=initial_b, name='B')

        layer_in = tf.matmul(_input, w) + b
        
        if name=='out':
            layer_output = tf.nn.softmax(layer_in)
        else:
            layer_output = tf.nn.relu(layer_in)

        tf.summary.histogram('weights', w)
        tf.summary.histogram('biases', b)
        
        return layer_output

In [10]:
layer_1 = setup_layer(x, weight=[TOTAL_INPUTS, n_hidden1], bias=[n_hidden1], name='layer_1')

layer_drop = tf.nn.dropout(layer_1, keep_prob=0.8, name='dropout_layer')

layer_2 = setup_layer(layer_drop, weight=[n_hidden1, n_hidden2], bias=[n_hidden2], name='layer_2')

output = setup_layer(layer_2, weight=[n_hidden2, NR_CLASSES], bias=[NR_CLASSES], name='out')

model_name = f'{n_hidden1}-{n_hidden2} LR{learning_rate} E{nr_epochs}'

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


# Tensorboard Setup

In [11]:
folder_name = f'{model_name} at {strftime("%H:%M")}'
directory = os.path.join(LOGGING_PATH, folder_name)

try:
    os.makedirs(directory)
except OSError as exception:
    print(exception.strerror)
else:
    print('Successfully created directories!')

Successfully created directories!


# Loss, Optimization and Metrics

In [12]:
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=output))

In [13]:
with tf.name_scope('optmizer'):
    optmizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    train_step = optmizer.minimize(loss=loss)

In [14]:
with tf.name_scope('accuracy_calc'):
    model_prediction = tf.argmax(output, axis=1, name='prediction')
    correct_pred = tf.equal(model_prediction, tf.argmax(y, axis=1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [15]:
with tf.name_scope('performance'):
    tf.summary.scalar('accuracy', accuracy)
    tf.summary.scalar('cost', loss)

In [16]:
with tf.name_scope('show__image'):
    x_image = tf.reshape(x, [-1, 28, 28, 1])
    tf.summary.image('image_input', x_image, max_outputs=6)

# Run Session

In [17]:
sess = tf.Session()

2022-12-15 11:02:34.089157: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [18]:
# Setup Filewriter and Merge Summaries
merged_summary = tf.summary.merge_all()

train_writer = tf.summary.FileWriter(directory + '/train')
train_writer.add_graph(sess.graph)

validation_writer = tf.summary.FileWriter(directory + '/validation')

In [19]:
init = tf.global_variables_initializer()
sess.run(init)

2022-12-15 11:02:34.113122: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled


In [20]:
# Batching
size_of_batch = 1000

In [21]:
num_examples = y_train.shape[0]
nr_iterations = int(num_examples / size_of_batch)

index_in_epoch = 0

In [22]:
def next_batch(batch_size, data, labels):
    global num_examples
    global index_in_epoch
    
    start = index_in_epoch
    index_in_epoch += batch_size
    
    if index_in_epoch > num_examples:
        start = 0
        index_in_epoch = batch_size
    
    end = index_in_epoch
    
    return data[start:end], labels[start:end]

### Training Loop

In [23]:
for epoch in range(nr_epochs):
    #Training dataset
    for i in range(nr_iterations):
        batch_x, batch_y = next_batch(batch_size=size_of_batch, data=x_train, labels=y_train)
        
        feed_dictionary = {x: batch_x, y: batch_y}
        sess.run(train_step, feed_dict=feed_dictionary)
        
    s, batch_accuracy = sess.run(fetches=[merged_summary, accuracy], feed_dict=feed_dictionary)
    
    train_writer.add_summary(s, epoch)
    
    print(f'Epoch {epoch} \t| Training Accuracy = {batch_accuracy}')
    
    #Valdation dataset
    summary = sess.run(fetches=merged_summary, feed_dict={x: x_val, y: y_val})
    validation_writer.add_summary(summary, epoch)
print('Done training!')

Epoch 0 	| Training Accuracy = 0.3869999945163727
Epoch 1 	| Training Accuracy = 0.6919999718666077
Epoch 2 	| Training Accuracy = 0.800000011920929
Epoch 3 	| Training Accuracy = 0.8240000009536743
Epoch 4 	| Training Accuracy = 0.8429999947547913
Epoch 5 	| Training Accuracy = 0.8529999852180481
Epoch 6 	| Training Accuracy = 0.8500000238418579
Epoch 7 	| Training Accuracy = 0.8489999771118164
Epoch 8 	| Training Accuracy = 0.8560000061988831
Epoch 9 	| Training Accuracy = 0.8560000061988831
Epoch 10 	| Training Accuracy = 0.8539999723434448
Epoch 11 	| Training Accuracy = 0.8600000143051147
Epoch 12 	| Training Accuracy = 0.8610000014305115
Epoch 13 	| Training Accuracy = 0.8600000143051147
Epoch 14 	| Training Accuracy = 0.8640000224113464
Epoch 15 	| Training Accuracy = 0.8659999966621399
Epoch 16 	| Training Accuracy = 0.8619999885559082
Epoch 17 	| Training Accuracy = 0.8669999837875366
Epoch 18 	| Training Accuracy = 0.8650000095367432
Epoch 19 	| Training Accuracy = 0.86599999

# Saving the Model

In [33]:
outputs = {'accuracy_calc/prediction': model_prediction}
inputs = {'x', x}

tf.compat.v1.saved_model.simple_save(sess, 'SavedModel', inputs, outputs, legacy_init_op=None)

AttributeError: 'set' object has no attribute 'items'

# Reset for the next run

In [ ]:
from PIL import Image

In [ ]:
img = Image.open('data/test_img.png')

In [ ]:
bw = img.convert(mode='L')

In [ ]:
test_img = np.invert(bw).ravel()

In [ ]:
prediction = sess.run(fetches=tf.argmax(output, axis=1), feed_dict={x: [test_img]})
prediction

In [ ]:
train_writer.close()
validation_writer.close()
sess.close()
tf.reset_default_graph()